In [11]:
from snowflake.snowpark.session import Session

connection_params = {'user': "REFRACT.FOSFOR@LNTINFOTECH.COM", 'password': "Password321", 'account':"fya62509.us-east-1", 'role':"FOSFOR_REFRACT", 'warehouse': "FOSFOR_REFRACT"}

session = Session.builder.configs(connection_params).create()
session.sql("USE FOSFOR_REFRACT;").collect()

#data = session.sql("SHOW TABLES;").collect()
#print(data)

#data= session.sql("SELECT * from BIAS.BIASED_TABLE limit 1;").collect()
#print(data)

session.sql('CREATE OR REPLACE TABLE BIAS.sample_product_data (id INT, parent_id INT, category_id INT, name VARCHAR, serial_number VARCHAR, key INT, "3rd" INT);').collect()
r1=session.sql("""
    INSERT INTO BIAS.sample_product_data VALUES
    (1, 0, 5, 'Product 1', 'prod-1', 1, 10),
    (2, 1, 5, 'Product 1A', 'prod-1-A', 1, 20),
    (3, 1, 5, 'Product 1B', 'prod-1-B', 1, 30),
    (4, 0, 10, 'Product 2', 'prod-2', 2, 40),
    (5, 4, 10, 'Product 2A', 'prod-2-A', 2, 50)
    """).collect()
print(r1)

session.close()

[Row(number of rows inserted=5)]


In [12]:
from snowflake.snowpark.functions import col

session = Session.builder.configs(connection_params).create()
session.sql("USE FOSFOR_REFRACT;").collect()

df_table = session.table("BIAS.sample_product_data")
df_table1 = session.table("BIAS.sample_product_data").select(col("id"), col("name"), col("serial_number"))
#df1 = session.create_dataframe([1, 2, 3, 4]).to_df("a")
df_table.show()
df_table1.show()


session.close()

-------------------------------------------------------------------------------------
|"ID"  |"PARENT_ID"  |"CATEGORY_ID"  |"NAME"      |"SERIAL_NUMBER"  |"KEY"  |"3rd"  |
-------------------------------------------------------------------------------------
|1     |0            |5              |Product 1   |prod-1           |1      |10     |
|2     |1            |5              |Product 1A  |prod-1-A         |1      |20     |
|3     |1            |5              |Product 1B  |prod-1-B         |1      |30     |
|4     |0            |10             |Product 2   |prod-2           |2      |40     |
|5     |4            |10             |Product 2A  |prod-2-A         |2      |50     |
-------------------------------------------------------------------------------------

---------------------------------------
|"ID"  |"NAME"      |"SERIAL_NUMBER"  |
---------------------------------------
|1     |Product 1   |prod-1           |
|2     |Product 1A  |prod-1-A         |
|3     |Product 1B  |prod

In [31]:
from snowflake.snowpark.types import IntegerType
from snowflake.snowpark.functions import udf

session = Session.builder.configs(connection_params).create()
session.sql("USE FOSFOR_REFRACT;").collect()

_ = session.sql("create or replace temp stage BIAS.mystage;").collect()
_ = session.udf.register(
    lambda x, y: x * y, return_type=IntegerType(),
    input_types=[IntegerType(), IntegerType()],
    is_permanent=True, name="mul", replace=True,
    stage_location="BIAS.mystage",
)
session.sql("select mul(5, 6) as mul").collect()



Failed to execute query [queryID: 01b2f557-0604-6722-0072-f3030e796c22] 
CREATE OR REPLACE 
  FUNCTION mul(arg1 INT,arg2 INT)
RETURNS INT
LANGUAGE PYTHON 
RUNTIME_VERSION=3.8

PACKAGES=('cloudpickle==2.0.0')
HANDLER='compute'

AS $$
import pickle

func = pickle.loads(bytes.fromhex('800595c1010000000000008c17636c6f75647069636b6c652e636c6f75647069636b6c65948c0d5f6275696c74696e5f747970659493948c0a4c616d6264615479706594859452942868028c08436f6465547970659485945294284b024b004b004b024b024b4343087c007c0114005300944e8594298c0178948c01799486948c1f3c69707974686f6e2d696e7075742d33312d3934313734613836363465373e948c083c6c616d6264613e944b094300942929749452947d94288c0b5f5f7061636b6167655f5f944e8c085f5f6e616d655f5f948c085f5f6d61696e5f5f94754e4e4e749452948c1c636c6f75647069636b6c652e636c6f75647069636b6c655f66617374948c125f66756e6374696f6e5f736574737461746594939468187d947d94286815680f8c0c5f5f7175616c6e616d655f5f94680f8c0f5f5f616e6e6f746174696f6e735f5f947d948c0e5f5f6b7764656661756c74735f5f944e8c0c5f5f64656

SnowparkSQLException: (1304): 01b2f557-0604-6722-0072-f3030e796c22: 090106 (22000): Cannot perform CREATE FUNCTION. This session does not have a current schema. Call 'USE SCHEMA', or use a qualified name.